# Beijing Air Quality Time Series Forecasting

In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

In [ ]:
import os

# Define the path to the data folder
data_folder = 'data'

# List all CSV files in the data folder
csv_files = [f for f in os.listdir(data_folder) if f.endswith('.csv')]

# Initialize an empty list to store dataframes
dataframes = []

# Loop through the CSV files and read them into dataframes
for file in csv_files:
    file_path = os.path.join(data_folder, file)
    df = pd.read_csv(file_path)
    dataframes.append(df)

# Concatenate all dataframes into a single dataframe
all_data = pd.concat(dataframes)
all_data.drop(columns=['Unnamed: 0', 'No'], inplace=True, errors='ignore')

# Create a new column 'date' by parsing year, month, and day columns
all_data['date'] = pd.to_datetime(all_data[['year', 'month', 'day']])

# Display the first few rows of the consolidated all_dataframe
all_data.head()

## EDA

In [ ]:
# Data Preprocessing
# Check for missing values
print(all_data.isnull().sum())

# Fill missing values (if any)
all_data.fillna(method='ffill', inplace=True)

# Visualize the all_data
# Plot all_data for different stations separately
stations = all_data['station'].unique()
for station in stations:
    plt.figure(figsize=(15, 2))
    station_data = all_data[all_data['station'] == station]
    plt.plot(station_data['date'], station_data['PM2.5'], label=station)
    plt.title(f'Beijing Air Quality - PM2.5 Levels at {station}')
    plt.xlabel('Date')
    plt.ylabel('PM2.5')
    plt.legend()
    plt.show()

In [ ]:
from ydata_profiling import ProfileReport

# Filter data to a station
data = all_data[all_data['station'] == 'Dongsi']

#Enable tsmode to True to automatically identify time-series variables
#Provide the column name that provides the chronological order of your time-series
profile = ProfileReport(data, tsmode=True, sortby="date", title="Time-Series EDA")

profile.to_file("report_timeseries.html")
# profile.to_widgets()

## Data Preprocessing

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

# One hot encode the 'wd' column
data = pd.get_dummies(data, columns=['wd'], dtype=int)

# Group the data by 'date' and calculate the mean
data = data.drop(columns=['station']).groupby('date').agg(['mean']).reset_index()
data.columns = data.columns.droplevel(1)

# Decompose the data
decomposition = seasonal_decompose(data['PM2.5'].ffill(), model='additive', period=30)

# Plot the decomposed components
fig, (ax1, ax2, ax3, ax4) = plt.subplots(nrows=4, sharex=True, figsize=(15, 5))
ax1.plot(data.date, decomposition.observed, label='Observed')
ax1.legend(loc='upper left')

ax2.plot(data.date, decomposition.trend, label='Trend')
ax2.legend(loc='upper left')

ax3.plot(data.date, decomposition.seasonal, label='Seasonality')
ax3.legend(loc='upper left')

ax4.plot(data.date, decomposition.resid, label='Residuals')
ax4.legend(loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller

input_data = data['PM2.5'].ffill().dropna()

# Perform Augmented Dickey-Fuller test
adf_result = adfuller(input_data)
print('ADF Statistic:', adf_result[0])
print('p-value:', adf_result[1])
for key, value in adf_result[4].items():
    print('Critical Value (%s): %.3f' % (key, value))

# Plot ACF and PACF
lag = 50
plt.figure(figsize=(12, 6))
plt.subplot(121)
plot_acf(input_data, lags=lag, ax=plt.gca())
plt.title('Autocorrelation Function (ACF)')

plt.subplot(122)
plot_pacf(input_data, lags=lag, ax=plt.gca())
plt.title('Partial Autocorrelation Function (PACF)')
plt.show()


In [7]:
# Define the model parameters

p, d, q = 2, 0, 4

## ARIMA

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

# ARIMA Model
# Split the data into training and testing sets
train = data[data['date'] < '2016-06-01']
test = data[data['date'] >= '2016-06-01'].ffill()

# Fit the ARIMA model
arima_model = ARIMA(train['PM2.5'], order=(p, d, q))
arima_result = arima_model.fit()

# Forecast
arima_forecast = arima_result.forecast(steps=len(test))
test['ARIMA_Forecast'] = arima_forecast

# Calculate MAE and MAPE
mae = mean_absolute_error(test['PM2.5'], test['ARIMA_Forecast'].ffill())
mape = mean_absolute_percentage_error(test['PM2.5'], test['ARIMA_Forecast'].ffill())

# Plot the forecast
plt.figure(figsize=(18, 3))
plt.plot(train.date, train['PM2.5'], label='Train')
plt.plot(test.date, test['PM2.5'], label='Test')
plt.plot(test.date, test['ARIMA_Forecast'], label='ARIMA Forecast: \nMAE=%.2f, MAPE=%.2f' % (mae, mape))
plt.title('ARIMA Forecast vs Actual')
plt.xlabel('Date')
plt.ylabel('PM2.5')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

In [ ]:
from ipywidgets import widgets, interact

import matplotlib.pyplot as plt

# Define a function to plot the data with a given range
@interact(
        start=widgets.IntSlider(min=0, max=len(data)-100, step=10, value=0),
        window=widgets.IntSlider(min=10, max=len(test), step=50, value=len(test))
)
def plot_data(start, window):
    plt.figure(figsize=(18, 3))
    plt.plot(train.date[start:], train['PM2.5'][start:], label='Train')
    plt.plot(test.date[:window], test['PM2.5'][:window], label='Test')
    plt.plot(test.date[:window], test['ARIMA_Forecast'][:window], label='ARIMA Forecast')
    plt.xlabel('Date')
    plt.ylabel('PM2.5')
    plt.title('PM2.5 Levels Over Time')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.show()


## ARIMAX

In [ ]:
# ARIMAX Model (Univariate Time Series Analysis with Exogenous Variables)
# Assuming we have additional exogenous variables like temperature and humidity in the dataset

from statsmodels.tsa.statespace.sarimax import SARIMAX

# Split the data into training and testing sets for ARIMAX model
exog_cols = ['TEMP', 'PRES', 'DEWP', 'WSPM', 'SO2', 'NO2', 'CO', 'O3'] + [col for col in train.columns if 'wd' in col]
exog_cols = ['TEMP', 'PRES', 'DEWP', 'WSPM', 'SO2', 'NO2', 'CO', 'O3'] + [col for col in train.columns if 'wd' in col]
steps = 365
train_exog = train[exog_cols].ffill()
test_exog = train_exog[-steps:]

# Fit the ARIMAX model
arimax_model = SARIMAX(train['PM2.5'], exog=train_exog, order=(p, d, q))
arimax_result = arimax_model.fit()

# Forecast with ARIMAX model
arimax_forecast = arimax_result.forecast(steps=365, exog=test_exog)
test['ARIMAX_Forecast'] = arimax_forecast.clip(0)

# Calculate MAE and MAPE
mae = mean_absolute_error(test['PM2.5'], test['ARIMAX_Forecast'].ffill())
mape = mean_absolute_percentage_error(test['PM2.5'], test['ARIMAX_Forecast'].ffill())

# Plot the ARIMAX forecast
plt.figure(figsize=(18, 3))
plt.plot(train.date, train['PM2.5'], label='Train')
plt.plot(test.date, test['PM2.5'], label='Test')
plt.plot(test.date, test['ARIMAX_Forecast'], label='ARIMAX Forecast: \nMAE=%.2f, MAPE=%.2f' % (mae, mape))
plt.title('ARIMAX Forecast vs Actual')
plt.xlabel('Date')
plt.ylabel('PM2.5')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

In [ ]:
from ipywidgets import widgets, interact

import matplotlib.pyplot as plt

# Define a function to plot the data with a given range
@interact(
        start=widgets.IntSlider(min=0, max=len(data)-100, step=10, value=0),
        window=widgets.IntSlider(min=10, max=len(test), step=50, value=len(test))
)
def plot_data(start, window):
    plt.figure(figsize=(18, 3))
    plt.plot(train.date[start:], train['PM2.5'][start:], label='Train')
    plt.plot(test.date[:window], test['PM2.5'][:window], label='Test')
    plt.plot(test.date[:window], test['ARIMAX_Forecast'][:window], label='ARIMAX Forecast')
    plt.xlabel('Date')
    plt.ylabel('PM2.5')
    plt.title('PM2.5 Levels Over Time')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.show()


## SARIMAX

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

# SARIMAX Model (Multivariate Time Series Analysis)
# Assuming we have additional exogenous variables like temperature and humidity in the dataset

# Split the data into training and testing sets for SARIMAX model
steps = 365
train_exog = train[exog_cols].ffill()
test_exog = train_exog[-steps:]

# Fit the SARIMAX model
sarimax_model = SARIMAX(train['PM2.5'], exog=train_exog, order=(p, d, q), seasonal_order=(1, 1, 1, 7))
sarimax_result = sarimax_model.fit()

# Forecast with SARIMAX model
sarimax_forecast = sarimax_result.forecast(steps=steps, exog=test_exog)
test['SARIMAX_Forecast'] = sarimax_forecast.clip(0)

# Calculate MAE and MAPE
mae = mean_absolute_error(test['PM2.5'], test['SARIMAX_Forecast'].ffill())
mape = mean_absolute_percentage_error(test['PM2.5'], test['SARIMAX_Forecast'].ffill())

# Plot the SARIMAX forecast
plt.figure(figsize=(18, 3))
plt.plot(train.date, train['PM2.5'], label='Train')
plt.plot(test.date, test['PM2.5'], label='Test')
plt.plot(test.date, test['SARIMAX_Forecast'], label='SARIMAX Forecast: \nMAE=%.2f, MAPE=%.2f' % (mae, mape))
plt.title('SARIMAX Forecast vs Actual')
plt.xlabel('Date')
plt.ylabel('PM2.5')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

In [ ]:
from ipywidgets import widgets, interact

import matplotlib.pyplot as plt

# Define a function to plot the data with a given range
@interact(
        start=widgets.IntSlider(min=0, max=len(data)-100, step=10, value=0),
        window=widgets.IntSlider(min=10, max=len(test), step=50, value=len(test))
)
def plot_data(start, window):
    plt.figure(figsize=(18, 3))
    plt.plot(train.date[start:], train['PM2.5'][start:], label='Train')
    plt.plot(test.date[:window], test['PM2.5'][:window], label='Test')
    plt.plot(test.date[:window], test['SARIMAX_Forecast'][:window], label='SARIMAX Forecast')
    plt.xlabel('Date')
    plt.ylabel('PM2.5')
    plt.title('PM2.5 Levels Over Time')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.show()


## Vector Autoregression (VAR) Model

In [ ]:
from statsmodels.tsa.stattools import grangercausalitytests

# Perform Granger's causality test
maxlag=12
def grangers_causation_matrix(data, variables, test='ssr_chi2test'):    
    """Check Granger Causality of all possible combinations of the Time series.
    The rows are the response variable, columns are predictors. The values in the table 
    are the P-Values. P-Values lesser than the significance level (0.05), implies 
    the Null Hypothesis that the coefficients of the corresponding past values is 
    zero, that is, the X does not cause Y can be rejected.

    data      : pandas dataframe containing the time series variables
    variables : list containing names of the time series variables.
    """
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value

    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    return df

columns = ['PM2.5'] + exog_cols
grangers_causality_result = grangers_causation_matrix(data[columns].dropna(), variables=columns)

In [ ]:
grangers_causality_result

In [ ]:
from statsmodels.tsa.api import VAR

# Prepare the data for VAR model
new_exog_cols = [col for col in exog_cols if 'wd' not in col]
var_data = data[['date', 'PM2.5'] + new_exog_cols].ffill()
var_data.set_index('date', inplace=True, drop=True)

# Split the data into training and testing sets
train_var = var_data[var_data.index < '2016-06-01']
test_var = var_data[var_data.index >= '2016-06-01']

# Fit the VAR model
var_model = VAR(train_var)
var_result = var_model.fit(maxlags=15, ic='aic')

# Forecast
lag_order = var_result.k_ar
var_forecast_input = train_var.values[-lag_order:]
var_forecast = var_result.forecast(y=var_forecast_input, steps=len(test_var))

# Convert forecast to DataFrame
var_forecast_df = pd.DataFrame(var_forecast, index=test_var.index, columns=train_var.columns)

# Calculate MAE and MAPE
mae = mean_absolute_error(test_var['PM2.5'], var_forecast_df['PM2.5'].ffill())
mape = mean_absolute_percentage_error(test_var['PM2.5'], var_forecast_df['PM2.5'].ffill())

# Plot the forecast
plt.figure(figsize=(18, 3))
plt.plot(train_var['PM2.5'], label='Train')
plt.plot(test_var['PM2.5'], label='Test')
plt.plot(var_forecast_df['PM2.5'], label='VAR Forecast: \nMAE=%.2f, MAPE=%.2f' % (mae, mape))
plt.title('VAR Forecast vs Actual')
plt.xlabel('Date')
plt.ylabel('PM2.5')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

In [ ]:
var_forecast_df

In [ ]:
from ipywidgets import widgets, interact

import matplotlib.pyplot as plt

# Define a function to plot the data with a given range
@interact(
        start=widgets.IntSlider(min=0, max=len(data)-100, step=10, value=0),
        window=widgets.IntSlider(min=10, max=len(test), step=50, value=len(test))
)
def plot_data(start, window):
    plt.figure(figsize=(18, 3))
    plt.plot(train_var.index[start:], train_var['PM2.5'][start:], label='Train')
    plt.plot(test_var.index[:window], test_var['PM2.5'][:window], label='Test')
    plt.plot(test_var.index[:window], var_forecast_df['PM2.5'][:window], label='VAR Forecast')
    plt.ylabel('PM2.5')
    plt.title('PM2.5 Levels Over Time')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.show()


## Prophet Model

In [ ]:
from prophet import Prophet

# Prophet Model
# Prepare the data for Prophet
prophet_data = train.rename(columns={'date': 'ds', 'PM2.5': 'y'})

# Fit the Prophet model
prophet_model = Prophet()
prophet_model.fit(prophet_data)

# Make future dataframe
period = 365
future = prophet_model.make_future_dataframe(periods=period)
forecast = prophet_model.predict(future)

# Calculate MAE and MAPE
mae = mean_absolute_error(test['PM2.5'], forecast['yhat'].tail(len(test)))
mape = mean_absolute_percentage_error(test['PM2.5'], forecast['yhat'].tail(len(test)))

# Plot the forecast
fig = prophet_model.plot(forecast)
plt.title('Prophet Forecast vs Actual: MAE=%.2f, MAPE=%.2f' % (mae, mape))
plt.xlabel('Date')
plt.ylabel('PM2.5')
plt.show()

# Display the forecast components
fig2 = prophet_model.plot_components(forecast)
plt.show()

In [ ]:
from ipywidgets import widgets, interact

import matplotlib.pyplot as plt

# Define a function to plot the data with a given range
@interact(
        start=widgets.IntSlider(min=0, max=len(data)-100, step=10, value=0),
        window=widgets.IntSlider(min=10, max=len(test), step=50, value=len(test))
)
def plot_data(start, window):
    plt.figure(figsize=(18, 3))
    plt.plot(train_var.index[start:], train_var['PM2.5'][start:], label='Train')
    plt.plot(test_var.index[:window], test_var['PM2.5'][:window], label='Test')
    plt.plot(test_var.index[:window], forecast['yhat'].tail(window), label='VAR Forecast')
    plt.ylabel('PM2.5')
    plt.title('PM2.5 Levels Over Time')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.show()
